In [23]:
pip install pandas transformers torch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd

# Load the data
file_path = '/Users/arhan/Documents/usa/ddp/facebook_engagement_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())

# Check for missing values and general information
print(data.info())


            source_id         source_name              author  \
0             reuters             Reuters   Reuters Editorial   
1     the-irish-times     The Irish Times  Eoin Burke-Kennedy   
2     the-irish-times     The Irish Times   Deirdre McQuillan   
3  al-jazeera-english  Al Jazeera English          Al Jazeera   
4            bbc-news            BBC News            BBC News   

                                               title  \
0  NTSB says Autopilot engaged in 2018 California...   
1       Unemployment falls to post-crash low of 5.2%   
2  "Louise Kennedy AW2019: Long coats, sparkling ...   
3  North Korean footballer Han joins Italian gian...   
4  UK government lawyer says proroguing parliamen...   

                                         description  \
0  "The National Transportation Safety Board said...   
1  Latest monthly figures reflect continued growt...   
2  Autumn-winter collection features designer’s g...   
3  Han is the first North Korean player in the S

In [25]:
# Print all column names in the DataFrame
print(data.columns)

Index(['source_id', 'source_name', 'author', 'title', 'description', 'url',
       'url_to_image', 'published_at', 'content', 'top_article',
       'engagement_reaction_count', 'engagement_comment_count',
       'engagement_share_count', 'engagement_comment_plugin_count'],
      dtype='object')


In [26]:
# Calculate a total engagement score by summing relevant metrics
data['total_engagement'] = data['engagement_reaction_count'] + data['engagement_comment_count'] + data['engagement_share_count'] + data['engagement_comment_plugin_count']

# Define a threshold for good engagement
threshold = data['total_engagement'].quantile(0.75)  # Using the 75th percentile as the threshold

# Create a new binary column based on this threshold
data['label'] = (data['total_engagement'] >= threshold).astype(int)

In [27]:
from transformers import BertTokenizer

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Ensure the content column is a string and replace NaN values
data['content'] = data['content'].fillna('').astype(str)

# Tokenize text
def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Apply tokenization
data['input_ids'] = data['content'].apply(lambda x: tokenize_function(x)['input_ids'].squeeze(0))

In [28]:
from sklearn.model_selection import train_test_split

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(data['input_ids'], data['label'], test_size=0.2)

In [29]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize data
def tokenize_data(data):
    return tokenizer(data, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Tokenize the entire dataset
tokenized_texts = tokenize_data(data['content'].tolist())

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    tokenized_texts['input_ids'], data['label'], test_size=0.2
)

# Ensure labels are also tensors
train_labels = torch.tensor(train_labels.values)
val_labels = torch.tensor(val_labels.values)

# Prepare datasets using the correct PyTorch dataset and DataLoader utilities
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_texts, train_labels)
val_dataset = TensorDataset(val_texts, val_labels)

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [30]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [31]:
from sklearn.model_selection import train_test_split
import numpy as np

# Tokenize the entire dataset
tokenized_texts = tokenizer(data['content'].tolist(), padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Generate indices and split them
indices = np.arange(len(data['label']))
train_indices, val_indices = train_test_split(indices, test_size=0.2)

# Use the indices to create training and validation datasets
train_texts = {key: val[train_indices] for key, val in tokenized_texts.items()}
val_texts = {key: val[val_indices] for key, val in tokenized_texts.items()}
train_labels = data['label'].values[train_indices]
val_labels = data['label'].values[val_indices]

# Create instances of the CustomDataset
train_dataset = CustomDataset(train_texts, train_labels)
val_dataset = CustomDataset(val_texts, val_labels)

In [33]:
from transformers import BertForSequenceClassification

# Load BERT for sequence classification with two labels (good or bad engagement)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',  # Make sure this directory is accessible and writable
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="steps",  # This will save the model at each logging step; consider adjusting if too frequent
    load_best_model_at_end=True,
    save_total_limit=1  # Optionally add this to limit the number of saved models
)

In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset              # evaluation dataset
)

In [44]:
trainer.train()

  0%|          | 0/429 [00:00<?, ?it/s]

/var/folders/30/3s3tphn117b7hrs3h7xw1pr80000gn/T/ipykernel_58758/1591585656.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'loss': 0.3761, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.07}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.4884822368621826, 'eval_runtime': 2.2116, 'eval_samples_per_second': 129.317, 'eval_steps_per_second': 8.139, 'epoch': 0.07}
{'loss': 0.4179, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.14}


...

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.1578338146209717, 'eval_runtime': 2.2028, 'eval_samples_per_second': 129.833, 'eval_steps_per_second': 8.171, 'epoch': 2.87}
{'loss': 0.364, 'learning_rate': 4.2e-05, 'epoch': 2.94}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.1692448854446411, 'eval_runtime': 2.2014, 'eval_samples_per_second': 129.917, 'eval_steps_per_second': 8.177, 'epoch': 2.94}
{'train_runtime': 248.3854, 'train_samples_per_second': 13.793, 'train_steps_per_second': 1.727, 'train_loss': 0.23953617244333655, 'epoch': 3.0}


TrainOutput(global_step=429, training_loss=0.23953617244333655, metrics={'train_runtime': 248.3854, 'train_samples_per_second': 13.793, 'train_steps_per_second': 1.727, 'train_loss': 0.23953617244333655, 'epoch': 3.0})

In [45]:
eval_results = trainer.evaluate()
print(eval_results)

/var/folders/30/3s3tphn117b7hrs3h7xw1pr80000gn/T/ipykernel_58758/1591585656.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 1.0741196870803833, 'eval_runtime': 2.9514, 'eval_samples_per_second': 96.902, 'eval_steps_per_second': 6.099, 'epoch': 3.0}


In [47]:
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json')

In [48]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the tokenizer and model from the saved directory
tokenizer = BertTokenizer.from_pretrained('./results')
model = BertForSequenceClassification.from_pretrained('./results')

In [49]:
def prepare_input(caption):
    # Tokenize the caption
    inputs = tokenizer(caption, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    return inputs

In [50]:
def evaluate_caption(caption):
    # Prepare the input
    inputs = prepare_input(caption)
    
    # Move the inputs to the same device as the model
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Evaluate the model (inference)
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        predictions = probabilities.argmax(dim=1).item()
    
    return "Good engagement" if predictions == 1 else "Bad engagement"

In [65]:
caption = "Bridging the gap between glam and inclusivity! Just like the cars seamlessly crossing the highway bridge, Fenty Beauty by Rihanna brings together unmatched shades for ALL skin tones. #InclusiveBeauty 🌈✨🚗"
result = evaluate_caption(caption)
print(result)

Bad engagement


In [52]:
def evaluate_caption_with_probabilities(caption):
    # Prepare the input
    inputs = prepare_input(caption)
    
    # Move the inputs to the same device as the model
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Evaluate the model (inference)
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        predicted_class = probabilities.argmax(dim=1).item()
        confidence = probabilities[0, predicted_class].item()

    return f"Caption classified as {'Good engagement' if predicted_class == 1 else 'Bad engagement'} with {confidence:.2f} confidence"


In [53]:
def evaluate_caption_with_top_probabilities(caption):
    # Prepare the input
    inputs = prepare_input(caption)
    
    # Move the inputs to the same device as the model
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Evaluate the model (inference)
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
        top_probabilities, top_classes = torch.topk(probabilities, 2)

    result = f"Probabilities: \n"
    for prob, cls in zip(top_probabilities[0], top_classes[0]):
        class_name = "Good engagement" if cls.item() == 1 else "Bad engagement"
        result += f"{class_name}: {prob.item():.2f}\n"
    
    return result


In [56]:
!pip install TextBlob
from textblob import TextBlob

def evaluate_caption_with_sentiment(caption):
    sentiment = TextBlob(caption).sentiment
    engagement = evaluate_caption(caption)
    return f"{engagement}. Sentiment polarity: {sentiment.polarity:.2f}, subjectivity: {sentiment.subjectivity:.2f}"


  Obtaining dependency information for TextBlob from https://files.pythonhosted.org/packages/02/07/5fd2945356dd839974d3a25de8a142dc37293c21315729a41e775b5f3569/textblob-0.18.0.post0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 8.8 MB/s eta 0:00:00a 0:00:01


In [57]:
from textblob import TextBlob

def evaluate_caption_with_sentiment(caption):
    sentiment = TextBlob(caption).sentiment
    engagement = evaluate_caption(caption)
    return f"{engagement}. Sentiment polarity: {sentiment.polarity:.2f}, subjectivity: {sentiment.subjectivity:.2f}"


In [67]:
result = evaluate_caption_with_probabilities(caption)
print(result)

Caption classified as Bad engagement with 0.98 confidence


In [68]:
result = evaluate_caption_with_top_probabilities(caption)
print(result)

Probabilities: 
Bad engagement: 0.98
Good engagement: 0.02



In [69]:
result = evaluate_caption_with_sentiment(caption)
print(result)

Bad engagement. Sentiment polarity: 0.10, subjectivity: 0.10
